# Reading Data from csv file and doing basic data manipulation

In [1]:
import pandas as pd
df = pd.read_csv("1.csv", index_col=0)

## Get data for individual Trips

In [39]:
for tripID in set(df['tripID'].values):
    df_trip = df[df['tripID']==tripID]

## Converting Raw ECompass data to engineering units

In [40]:
import numpy as np
# Convert Accelerometer reading to g units, 8bit data with a range of +-2g #
def convert_acc(x):
    x = int(x,16)
    if x>127:
        x = x-256
    return np.float64(x*2/128)

# Convert a single row of Accelerometer data to engineering units and return a df #
def convert_acc_row(row):
    # Initially the data was gathered without magnetometer, so check length for identification #
    data_list = []
    for i in range(0, len(row), 6):
        x = convert_acc(row[i:i+2])
        y = convert_acc(row[i+2:i+4])
        z = convert_acc(row[i+4:i+6])
        
        data_list.append([x,y,z])
    return pd.DataFrame(columns=['ax','ay','az'], data=data_list)

df_trip['accData'] = df_trip['accData'].apply(convert_acc_row)
# Drop the values for first second as the data is not collected completely #
df_trip = df_trip.reset_index(drop=True)
df_trip = df_trip.drop(0).reset_index(drop=True)

C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Calculating Rotation Matrix for vehicle

In [37]:
# We need data for the device when the vehicle is static #
[mx,my,mz] = df_trip[['mx', 'my', 'mz']][df_trip['speed'] <= 1].mean().tolist()
[x,y,z] = df_trip['accData'][df_trip['speed'] <= 1].iloc[0].mean().tolist()

import math
phi = math.atan(y/z)
theta = math.atan(-x/(y*math.sin(phi)+z*math.cos(phi)))
psi = math.atan((mz*math.sin(phi) - my*math.cos(phi))/(mx*math.cos(theta) + my*math.sin(theta)*math.sin(phi) + mz*math.sin(theta)*math.cos(phi)))

Rx = np.array([[1,0,0], [0,math.cos(phi),math.sin(phi)], [0,-math.sin(phi),math.cos(phi)]])
Ry = np.array([[math.cos(theta),0,-math.sin(theta)], [0,1,0], [math.sin(theta),0,math.cos(theta)]])
Rz = np.array([[math.cos(phi),math.sin(phi),0], [-math.sin(phi),math.cos(phi),0], [0,0,1]])

R = Rx.dot(Ry).dot(Rz)
R = np.linalg.inv(R)

### Rotate Accelerometer values

In [52]:
def rotate_acc(row):
    A = np.array([[row['ax']], [row['ay']], [row['az']]])
    B = np.dot(R,A)
    x,y,z = B[:,0]
    return [x,y,z]


acc_sample = df_trip['accData'].iloc[0]
acc_sample.apply(rotate_acc, axis=1)

,ax,ay,az
0,0.015834,0.029995,1.002595
1,0.013017,0.021109,1.022630
2,0.024872,0.006861,1.042350
3,0.021468,0.047767,0.962526
4,-0.036071,-0.023454,1.044706
5,-0.031269,-0.056458,1.055114
6,-0.019280,-0.023880,1.059064
7,0.042496,0.057212,1.006230
8,0.015113,0.032961,0.918507
9,-0.029017,-0.004697,1.054017


## Storing DataFrame in hdf file

In [28]:
df.to_hdf("data.hdf5", "first")

C:\Miniconda3\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['timeStamp', 'accData']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
